In [1]:
import pandas as pd
from ortools.sat.python import cp_model
from datetime import datetime, timedelta

In [2]:
import sys
sys.path.append('/home/cayena/Projeto_thefamily/TeacherScheduler/')

In [3]:
from streamlit_app.utils import replicate_row,base_selection,expand_rows,clean_data
from streamlit_app.teacher_alocation import TeacherScheduler

In [4]:
professores_raw = pd.read_excel('/home/cayena/Downloads/professores_final.xlsx')
professores_raw.columns = professores_raw.columns.astype(str)

aulas_raw = pd.read_excel('/home/cayena/Downloads/rotas_finais.xlsx')

In [5]:
aulas_simples,doub,tri=base_selection(aulas_raw)

# transformando aulas duplas/triplas em 2/3 linhas
aulas_duplicadas = expand_rows(doub, lambda row: replicate_row(row, times=2))
aulas_triplicadas = expand_rows(tri, lambda row: replicate_row(row, times=3))

# juntando todas as linhas
df_tratado = pd.concat([aulas_simples, aulas_duplicadas, aulas_triplicadas], ignore_index=True)

df_final = clean_data(df_tratado)

df_final.head()

,Grupo,Horário,Unidade,Dias da Semana,MODALIDADE,GRUPO,N Aulas,Teacher,STATUS,validacao,intenviso,horario_tratado
0,ANDRESSA VIP EAD - 1X,11:00:00,Vicentina,2ª,Inglês,VIP,1,NaN,ONLINE,-,0,1900-01-01 11:00:00
2,KIKO VIP P - 1X,08:00:00,Vicentina,5ª,Inglês,VIP - In Company,1,NaN,PRESENCIAL,BRUNO MORGILLO,0,1900-01-01 08:00:00
4,VIVIANE BENEFICIAR VIP EAD - 2X,13:30:00,Jardim,3ª,Inglês,VIP,2,NaN,ONLINE,-,0,1900-01-01 13:30:00
6,VIVIANE BENEFICIAR VIP EAD - 2X,13:30:00,Jardim,5ª,Inglês,VIP,2,NaN,ONLINE,-,0,1900-01-01 13:30:00
8,KIDS DAISY P - 2X,09:30:00,Jardim,3ª,Kids,Grupo,2,NaN,PRESENCIAL,CENKAY,0,1900-01-01 09:30:00


In [9]:
for g in df_final['Grupo'].unique():
    for i in professores_raw['TEACHER'].unique():
        time_class = df_final.loc[df_final['Grupo'] == g, 'Horário'].values[0]
        day_class = df_final.loc[df_final['Grupo'] == g, 'Dia da semana'].to_list()
        if professores_raw.loc[professores_raw['TEACHER'] == i, time_class].values[0] == 0:
            print(f'Professor {i} não pode dar aula no horário {time_class} para o grupo {g}')
            

Professor ALINE não pode dar aula no horário 11:00:00 para o grupo ANDRESSA VIP EAD - 1X
Professor ANDREY não pode dar aula no horário 11:00:00 para o grupo ANDRESSA VIP EAD - 1X
Professor CARLOS não pode dar aula no horário 11:00:00 para o grupo ANDRESSA VIP EAD - 1X
Professor DANILO não pode dar aula no horário 11:00:00 para o grupo ANDRESSA VIP EAD - 1X
Professor DARLENE não pode dar aula no horário 11:00:00 para o grupo ANDRESSA VIP EAD - 1X
Professor DUDU não pode dar aula no horário 11:00:00 para o grupo ANDRESSA VIP EAD - 1X
Professor ERIK ERBEN não pode dar aula no horário 11:00:00 para o grupo ANDRESSA VIP EAD - 1X
Professor JULIANA não pode dar aula no horário 11:00:00 para o grupo ANDRESSA VIP EAD - 1X
Professor LARISSA ANDRADE não pode dar aula no horário 11:00:00 para o grupo ANDRESSA VIP EAD - 1X
Professor LEONARDO não pode dar aula no horário 11:00:00 para o grupo ANDRESSA VIP EAD - 1X
Professor LUCIANA não pode dar aula no horário 11:00:00 para o grupo ANDRESSA VIP EAD 

In [6]:
Ts = TeacherScheduler(df_final, professores_raw)
base_alocada = Ts.schedule_teachers()
base_alocada

,Professor,Grupo
0,NATACHA,ANDRESSA VIP EAD - 1X
1,DUDU,KIKO VIP P - 1X
2,ALINE,MBA MATRIZ EAD - 1X
3,ALINE,VIVIANE BENEFICIAR VIP EAD - 2X
4,ALINE,KIDS DAISY P - 2X
...,...,...
135,MARIA,BARCELONA P - 3X
136,LEONEL,GIBRALTAR P - 3X
137,DUDU,SCOTT EAD - 3X
138,TATIANA ERBEN,NORTHAMPTOM P - 3X
